In [28]:
import random
import pandas as pd

In [29]:
battery_expenditure = pd.read_csv('battery_expenditure_matrix.csv')
distance_matrix = pd.read_csv('distance_matrix.csv')
time_matrix = pd.read_csv('time_matrix.csv')

In [30]:
customer_demands = [[3,4],[5,15],[3,7],[8,5],[6,13]]

In [31]:
W1 = 0.5  # Adjust according to preference
W2 = 0.5  # Adjust according to preference
POPULATION_SIZE = 50
GENERATIONS = 100

In [32]:
def initialize_population(population_size, customer_demands=customer_demands):
    population = []
    for _ in range(population_size):
        # Generate random routes satisfying conditions
        route = [0]  # Start at node 0
        # Randomly shuffle the customer demands
        shuffled_demands = random.sample(customer_demands, len(customer_demands))
        for demand in shuffled_demands:
            route.append(demand[0])  # Start node of the customer demand
            route.append(demand[1])  # End node of the customer demand
        route.append(0)  # End at node 0
        population.append(route)
    return population

In [33]:
def evaluate_fitness(route):
    total_time = sum(time_matrix.iloc[route[i], route[i + 1]] for i in range(len(route) - 1))
    total_distance = sum(distance_matrix.iloc[route[i], route[i + 1]] for i in range(len(route) - 1))
    fitness = W1 * total_time + W2 * total_distance
    return fitness

In [34]:
def tournament_selection(population, k=5):
    participants = random.sample(population, k)
    winner = min(participants, key=evaluate_fitness)
    return winner

In [35]:
def ordered_crossover(parent1, parent2):
    # Perform ordered crossover on two parents
    start, end = sorted(random.sample(range(1, len(parent1) - 1), 2))
    offspring = [None] * len(parent1)
    
    # Copy the segment between start and end from parent1 to the offspring
    offspring[start:end] = parent1[start:end]
    
    # Fill in the remaining positions with genes from parent2
    remaining_positions = [i for i in range(1, len(parent1) - 1) if offspring[i] is None]
    index_parent2 = 0
    for position in remaining_positions:
        while parent2[index_parent2] in offspring[start:end]:
            index_parent2 += 1
        offspring[position] = parent2[index_parent2]
        index_parent2 += 1
    
    offspring[0] = 0
    offspring[-1] = 0
    return offspring

In [36]:
def shift_mutation(route):
    # Apply shift mutation to the route
    start, end = sorted(random.sample(range(1, len(route) - 1), 2))
    route[start:end] = route[start:end][::-1]
    return route

In [37]:
def replace_population(old_population, new_population):
    # Replace the old population with the new one based on fitness
    combined_population = old_population + new_population
    sorted_population = sorted(combined_population, key=evaluate_fitness)
    return sorted_population[:len(old_population)]

In [40]:

def genetic_algorithm():
    # Initialize population
    population = initialize_population(POPULATION_SIZE)
    print(population)
    for generation in range(GENERATIONS):
        # Evaluate initial population
        evaluated_population = [(route, evaluate_fitness(route)) for route in population]

        # Select individuals for crossover using tournament selection
        selected_population = [tournament_selection(evaluated_population) for _ in range(POPULATION_SIZE)]

        # Perform crossover to create new individuals
        new_population = []
        for i in range(0, POPULATION_SIZE, 2):
            parent1 = selected_population[i]
            parent2 = selected_population[i + 1]
            offspring1 = ordered_crossover(parent1, parent2)
            offspring2 = ordered_crossover(parent2, parent1)
            new_population.extend([offspring1, offspring2])

        # Apply mutation to the new individuals
        mutated_population = [shift_mutation(route) for route in new_population]

        # Evaluate the new population
        evaluated_mutated_population = [(route, evaluate_fitness(route)) for route in mutated_population]

        # Replace the old population with the new one
        population = replace_population(evaluated_population, evaluated_mutated_population)

    # Return the best route found
    best_route = min(population, key=evaluate_fitness)
    return best_route

In [39]:
best_route = genetic_algorithm()
print("Best Route:", best_route)

[[0, 8, 5, 6, 13, 3, 4, 3, 7, 5, 15, 0]]
[[0, 8, 5, 6, 13, 3, 4, 3, 7, 5, 15, 0], [0, 3, 4, 6, 13, 8, 5, 5, 15, 3, 7, 0]]
[[0, 8, 5, 6, 13, 3, 4, 3, 7, 5, 15, 0], [0, 3, 4, 6, 13, 8, 5, 5, 15, 3, 7, 0], [0, 3, 4, 3, 7, 5, 15, 6, 13, 8, 5, 0]]
[[0, 8, 5, 6, 13, 3, 4, 3, 7, 5, 15, 0], [0, 3, 4, 6, 13, 8, 5, 5, 15, 3, 7, 0], [0, 3, 4, 3, 7, 5, 15, 6, 13, 8, 5, 0], [0, 6, 13, 5, 15, 3, 4, 8, 5, 3, 7, 0]]
[[0, 8, 5, 6, 13, 3, 4, 3, 7, 5, 15, 0], [0, 3, 4, 6, 13, 8, 5, 5, 15, 3, 7, 0], [0, 3, 4, 3, 7, 5, 15, 6, 13, 8, 5, 0], [0, 6, 13, 5, 15, 3, 4, 8, 5, 3, 7, 0], [0, 6, 13, 3, 7, 8, 5, 5, 15, 3, 4, 0]]
[[0, 8, 5, 6, 13, 3, 4, 3, 7, 5, 15, 0], [0, 3, 4, 6, 13, 8, 5, 5, 15, 3, 7, 0], [0, 3, 4, 3, 7, 5, 15, 6, 13, 8, 5, 0], [0, 6, 13, 5, 15, 3, 4, 8, 5, 3, 7, 0], [0, 6, 13, 3, 7, 8, 5, 5, 15, 3, 4, 0], [0, 8, 5, 6, 13, 3, 7, 3, 4, 5, 15, 0]]
[[0, 8, 5, 6, 13, 3, 4, 3, 7, 5, 15, 0], [0, 3, 4, 6, 13, 8, 5, 5, 15, 3, 7, 0], [0, 3, 4, 3, 7, 5, 15, 6, 13, 8, 5, 0], [0, 6, 13, 5, 15, 3, 4, 8, 5, 3, 

ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types